## **Milestone 4: Baseline Pricing Engine**
### **Objective: Create a rule-based engine for initial comparison.**

In [5]:
import numpy as np
import pandas as pd

cleaned_df=pd.read_csv('cleaned_dynamic_pricing.csv')

In [6]:
def time_multiplier_map():
    return {
        "Morning": 1.10,
        "Afternoon": 1.00,
        "Evening": 1.10,
        "Night": 1.05
    }

def inventory_multiplier(riders, drivers):
    # Simple inventory-based pricing: increase when riders >> drivers, discount when drivers >>riders
    drivers = max(1, drivers) 
    ratio = riders / drivers
    if ratio > 2.0:
        return 1.15
    if ratio > 1.5:
        return 1.10
    if ratio < 0.7:
        return 0.90
    return 1.00

def apply_rule_based_pricing(df, elasticity=-0.5):
    """
    elasticity: price elasticity of demand (negative). Example -0.5 means 10% price increase -> 5% drop in quantity.
    """
    tmap = time_multiplier_map()
    df = df.copy()
    # baseline assumptions
    df['base_price'] = df['Historical_Cost_of_Ride']          
    df['base_qty'] = df['Number_of_Riders'].astype(float)  
    # To compute multipliers and new price
    df['time_mult'] = df['Time_of_Booking'].map(tmap).fillna(1.0) # default 1.0 if missing
    df['inv_mult'] = df.apply(lambda r: inventory_multiplier(r['Number_of_Riders'], r['Number_of_Drivers']), axis=1)
    df['price_mult'] = df['time_mult'] * df['inv_mult']
    df['dyn_price'] = df['base_price'] * df['price_mult']
    #  new_qty = base_qty * (dyn_price / base_price) ** elasticity
    df['price_ratio'] = df['dyn_price'] / df['base_price'].replace(0, np.nan) # to avoid division by zero
    df['qty_mult'] = df['price_ratio'].fillna(1.0) ** elasticity
    df['dyn_qty'] = df['base_qty'] * df['qty_mult']
    # revenues
    df['base_revenue'] = df['base_price'] * df['base_qty']
    df['dyn_revenue'] = df['dyn_price'] * df['dyn_qty']
    # aggregate metrics
    total_base_rev = df['base_revenue'].sum()
    total_dyn_rev = df['dyn_revenue'].sum()
    revenue_lift_pct = ((total_dyn_rev - total_base_rev) / total_base_rev) * 100 if total_base_rev != 0 else np.nan
    results = {
        'total_base_revenue': total_base_rev,
        'total_dynamic_revenue': total_dyn_rev,
        'revenue_lift_pct': revenue_lift_pct
    }
    return df, results

# Run the rule-based engine on the cleaned dataset
results_df, sim_metrics = apply_rule_based_pricing(cleaned_df, elasticity=-0.5)

print(f"Static total revenue: {sim_metrics['total_base_revenue']:.2f}")
print(f"Rule-based total revenue: {sim_metrics['total_dynamic_revenue']:.2f}")
print(f"Simulated revenue lift: {sim_metrics['revenue_lift_pct']:.2f}% (elasticity = -0.5)")

# Breakdown by Time_of_Booking to inspect where lift occurs
time_summary = results_df.groupby('Time_of_Booking')[['base_revenue', 'dyn_revenue']].sum()
time_summary['lift_pct'] = ((time_summary['dyn_revenue'] - time_summary['base_revenue']) / time_summary['base_revenue']) * 100
print("\nRevenue breakdown by Time_of_Booking:")
print(time_summary)

results_df.head()

Static total revenue: 22514545.02
Rule-based total revenue: 24408134.19
Simulated revenue lift: 8.41% (elasticity = -0.5)

Revenue breakdown by Time_of_Booking:
                 base_revenue   dyn_revenue   lift_pct
Time_of_Booking                                       
Afternoon        5.488150e+06  5.770574e+06   5.146068
Evening          5.203018e+06  5.750075e+06  10.514221
Morning          5.514224e+06  6.099820e+06  10.619745
Night            6.309153e+06  6.787665e+06   7.584408


,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,...,base_qty,time_mult,inv_mult,price_mult,dyn_price,price_ratio,qty_mult,dyn_qty,base_revenue,dyn_revenue
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273,...,90.0,1.05,1.10,1.155,328.317150,1.155,0.930484,83.743579,25583.154572,27494.453196
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753,...,58.0,1.10,1.00,1.100,191.262228,1.100,0.953463,55.300830,10084.735659,10576.959991
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469,...,42.0,1.00,1.00,1.000,329.795469,1.000,1.000000,42.000000,13851.409696,13851.409696
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232,...,89.0,1.00,1.15,1.150,540.731417,1.150,0.932505,82.992928,41847.909626,44876.883482
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422,...,78.0,1.00,1.15,1.150,666.633636,1.150,0.932505,72.735375,45215.150944,48487.847510
